In [69]:
!pip install nltk datasets transformers[torch]
!pip install tokenizers evaluate rouge_score sentencepiece
!pip install huggingface_hub

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [70]:
import nltk
import evaluate
import random
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset, DatasetDict, load_metric
import datasets
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [71]:
from datasets import load_dataset

dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [72]:
dataset = dataset.rename_column('Question', 'question')
dataset = dataset.rename_column('Answer', 'answer')
dataset

DatasetDict({
    train: Dataset({
        features: ['qtype', 'question', 'answer'],
        num_rows: 16407
    })
})

In [73]:
def reduce_data(raw_datasets, no_of_rows):

  shuffled_dataset = raw_datasets.shuffle(seed=42)
  reduced_data = shuffled_dataset.select(range(no_of_rows))

  return reduced_data

In [74]:
from IPython.display import display, HTML
import datasets
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [75]:
show_random_elements(dataset['train'])

,qtype,question,answer
0,symptoms,What are the symptoms of Heart Failure ?,"The most common symptoms of heart failure include shortness of breath or difficulty breathing, feeling tired, and swelling. Swelling is caused by fluid build-up in the body. Fluid buildup can lead to weight gain and frequent urination, as well as coughing."
1,treatment,What are the treatments for Opsoclonus Myoclonus ?,"Treatment for opsoclonus myoclonus may include corticosteroids or ACTH (adrenocorticotropic hormone). In cases where there is a tumor present, treatment such as chemotherapy, surgery, or radiation may be required."
2,treatment,What are the treatments for cardiofaciocutaneous syndrome ?,These resources address the diagnosis or management of cardiofaciocutaneous syndrome: - Gene Review: Gene Review: Cardiofaciocutaneous Syndrome - Genetic Testing Registry: Cardiofaciocutaneous syndrome These resources from MedlinePlus offer information about the diagnosis and management of various health conditions: - Diagnostic Tests - Drug Therapy - Surgery and Rehabilitation - Genetic Counseling - Palliative Care
3,frequency,How many people are affected by hand-foot-genital syndrome ?,Hand-foot-genital syndrome is very rare; only a few families with the condition have been reported worldwide.
4,frequency,How many people are affected by cleidocranial dysplasia ?,Cleidocranial dysplasia occurs in approximately 1 per million individuals worldwide.


In [76]:
dataset_split = dataset['train'].train_test_split(test_size=0.2)
val_test_split = dataset_split['test'].train_test_split(test_size=0.5)
# print(dataset_split)
# print(val_test_split)

final_dataset_dict = DatasetDict({
    # 'train': reduce_data(dataset_split['train'], 131),
    # 'validation': reduce_data(val_test_split['train'], 16),
    # 'test': reduce_data(val_test_split['test'], 16)
    'train': dataset_split['train'],
    'validation': val_test_split['train'],
    'test': val_test_split['test'],
})
final_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['qtype', 'question', 'answer'],
        num_rows: 13125
    })
    validation: Dataset({
        features: ['qtype', 'question', 'answer'],
        num_rows: 1641
    })
    test: Dataset({
        features: ['qtype', 'question', 'answer'],
        num_rows: 1641
    })
})

In [77]:
model_checkpoint = "google/flan-t5-base"

In [78]:
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model = model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


KeyboardInterrupt: 

In [79]:
max_input_length = 128
max_target_length = 256

prefix = "answer the question: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # edit: counting the avg length of input
    # print(model_inputs)
    # lens = [len(pred) for pred in model_inputs]
    # print("mean question length" , np.mean(lens))
    # print(examples["question"][0].split(' '))
    # lens_ = [len(pred.split(' ')) for pred in examples["question"]]
    # print("mean question length" , np.mean(lens_))

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["answer"], max_length=max_target_length, truncation=True)
    # labels = tokenizer(labels, max_length=max_target_length, truncation=True)

    # print(labels)
    # lens = [len(pred) for pred in labels]
    # print("mean answer length" , np.mean(lens))

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [80]:
tokenizer.pad_token_id

0

In [81]:
tokenized_datasets = final_dataset_dict.map(preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/13125 [00:00<?, ? examples/s]

Map:   0%|          | 0/1641 [00:00<?, ? examples/s]

Map:   0%|          | 0/1641 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['qtype', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 13125
    })
    validation: Dataset({
        features: ['qtype', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1641
    })
    test: Dataset({
        features: ['qtype', 'question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1641
    })
})

In [82]:
from evaluate import load
rouge_metric = load('rouge')

In [83]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}


    # Add mean generated length
    # print(predictions)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    print(result)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [84]:
batch_size = 3
no_of_train_epochs = 3
model_name = model_checkpoint.split("/")[-1]
learning_rate = 3e-4
per_device_eval_batch = 4

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    report_to=None,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=per_device_eval_batch,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=no_of_train_epochs,
    predict_with_generate=True,
    # push_to_hub=True,
)

In [85]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [86]:
torch.cuda.empty_cache()

In [87]:
trainer.train()
# NOTE: Rouge Scores are multiplied by 100

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.682500,1.537193,16.518400,10.610800,15.282500,15.953600,18.988400
2,1.522900,1.445218,16.673600,10.879700,15.400700,16.068100,18.904900
3,1.385300,1.417370,17.024300,11.212700,15.749500,16.422300,18.901300


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[[    0 17320    18 ...     3     9 16316]
 [    0   363    33 ...    32  6137   461]
 [    0  5586    15 ...   118  2196  4388]
 ...
 [    0  7112    15 ...     5    94    19]
 [    0   571    19 ...  1086 18024     3]
 [    0   571    19 ...   288 10207     9]]
{'rouge1': 16.518363024964238, 'rouge2': 10.610778050868529, 'rougeL': 15.282453858254385, 'rougeLsum': 15.953581532228853, 'gen_len': 18.98842169408897}


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[[    0 17320    18 ...     7     8   643]
 [    0   363    33 ...    32  6137   461]
 [    0  5586    15 ...     0     0     0]
 ...
 [    0  7112    15 ...  2241     5    94]
 [    0   571    19 ...    30     8  3053]
 [    0   571    19 ...  8209    13 22726]]
{'rouge1': 16.67355861815179, 'rouge2': 10.879682926945696, 'rougeL': 15.40072042577968, 'rougeLsum': 16.068084368440626, 'gen_len': 18.90493601462523}


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[[    0 17320    18 ...     7     8   643]
 [    0   363    33 ...    32  6137   461]
 [    0  5586    15 ...     0     0     0]
 ...
 [    0  7112    15 ...  2241     5    94]
 [    0   571    19 ...    30     8  3053]
 [    0   571    19 ...   288 10207     9]]
{'rouge1': 17.024291889941264, 'rouge2': 11.21266499047498, 'rougeL': 15.7495087859501, 'rougeLsum': 16.42232737486834, 'gen_len': 18.90127970749543}


TrainOutput(global_step=13125, training_loss=1.5594645368303572, metrics={'train_runtime': 6612.0382, 'train_samples_per_second': 5.955, 'train_steps_per_second': 1.985, 'total_flos': 1295138287153152.0, 'train_loss': 1.5594645368303572, 'epoch': 3.0})

In [91]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [93]:
model_checkpoint

'google/flan-t5-base'

In [94]:
import os

def save_model(model, tokenizer, model_path):
  if not os.path.exists(model_path):
    os.makedirs(model_path)
  print("Directory created for new checkpt to save")

  model.save_pretrained(model_path)
  tokenizer.save_pretrained(model_path)

model_name = model_checkpoint.split('/')[-1]
model_path = "/content/drive/MyDrive/Augnito/models/" + f"{model_name}-{batch_size}-{no_of_train_epochs}"
save_model(model, tokenizer, model_path)

Directory created for new checkpt to save
